In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pylab
import seaborn as sns
#plt.style.use('seaborn')
%matplotlib inline
pylab.rcParams['figure.figsize'] = (15, 10)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Кластеризация или обучение без учителя

Основное отличие методов обучения без учителя от классификации в том, что разметка для данных отсутствует. Задачи кластеризации возникают при снижении размерности задач, при обработке больших потоков данных (точность обработки при этом теряется, но скорость возрастает), при разработке отдельных моделей для каждого кластера и т.д.

Снижение размерности задач имеет ряд полезных свойств:
1. Увеличение скорости построения и работы классификатора;
2. Снижение зашумленности данных;
3. Снижение корреляции признаков;
4. Возможность визуализировать многомерные данные на плоскости и др.

Вначале познакомимся с методами кластеризации k-Means и агломеративным методом. Затем познакомимся с методами снижения размерности задач PCA и t-SNE. и на последок познакомимся с еще одним методом кластеризации DBSCAN. По каждому рассмотренному методу потребуется решить задание.

---

Общий принцип кластеризации: требуется разбить множество объектов на группы таким образом, чтобы элементы внутри одной группы (кластера) были похожи друг на друга, а элементы из разных групп отличались.

## k-Means

[Алгоритм к-средних](https://ru.wikipedia.org/wiki/Метод_k-средних) один из самых популярных и простых методов кластеризации. 

### Подготовка данных

Для реализации алгоритма вначале необходимо получить данные с помощью функции [make_blobs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) библиотеки sklearn:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=1000, n_features=5, centers=4, random_state=21)
X, y

---

Нормализуем полученные данные:

In [ ]:
X = (X - X.mean(axis=0))/X.std(axis=0)
X

---

С целью визуализации данных, запишем их в DataFrame и построим графики:

In [ ]:
df = pd.DataFrame(X, columns=['x0', 'x1', 'x2', 'x3', 'x4'])
df
# df['y'] = y
# y
# sns.pairplot(df, hue="y", height=3)
# plt.plot()

Наиболее хорошо кластеры разбиваются по переменным х2 и х4 - именно их мы будем использовать для визуализации на плоскости.

---

Данные готовы. 

### Обучение модели

Импортируем метод [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

Метод к-средних требует указать количество формируемых кластеров. Предположим, что мы не знаем количество кластеров, и укажем требование разбить на 2 кластера. Посмотрим на результаты:

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=21).fit(X)
plt.scatter(X[:,2], X[:,4], c=kmeans.labels_)
plt.show()

---

В результате три кластера были объединены в один.

---

Для определения количества кластеров в данных можно воспользоваться "методом локтя". Суть метода заключается в том, что строится множество моделей к-средних с возрастающим количеством кластеров. По каждой модели считается метрика $J(C_k)$, которая представляет собой корень из суммы квадратов расстояний от точек до центройдов кластеров, к которым они относятся (***kmeans.inertia_***). Выбирается то количество кластеров, при котором метрика резко уменьшает свое снижение (локоть):

In [ ]:
inertia = []
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=21).fit(X)
    inertia.append(np.sqrt(kmeans.inertia_))

plt.plot(range(2, 8), inertia, marker='s')
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$')
plt.show()

---

Методом локтя определили, что требуется 4 кластера:

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=21).fit(X)
plt.scatter(X[:,2], X[:,4], c=kmeans.labels_)
plt.show()

---

Как оценить качество кластеризации? Если разбиение на кластеры интуитивно понятно или имеются размеченные данные, то можно воспользоваться известными нам Precision, Recall и F-мерой, а в случае сбалансированности кластеров (в каждом кластере примерно одинаковое количество объектов), то можно воспользоваться Accuracy или его аналогом, [индексом Рэнда](https://en.wikipedia.org/wiki/Rand_index) - в библиотеке sklearn [adjusted_rand_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html)

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, kmeans.labels_)

1.0

---

Индекс Рэнда равен 1 - это значит, что разбиение на кластеры полностью соответствует классам в данных.

---

В случае отсутствия размеченных данных и интуитивного понимания разбиения на кластеры, потребуются иные метрики оценки качества кластеризации.

[silhouette_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html?highlight=silhouette_score#sklearn.metrics.silhouette_score) позволяет оценить кластеризацию с точки зрения близости объектов к центройду кластера, к которому принадлежат, и удаленность от кластеров, к которым не принадлежат. Недостатком метрики является то, что она хорошо работает только с кластерами круглой формы.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_score(X, kmeans.labels_, random_state=21)

0.7886615410756992

---

Близость silhouette_score к 1 говорит об идеальном разбиении на кластеры, к -1 - к идеально плохому разбиению.

Метрику silhouette_score можно применять для определения количества кластеров вместо метода локтя:

In [ ]:
silhouette = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=21).fit(X)
    silhouette.append(silhouette_score(X, km.labels_, random_state=21))

plt.plot(range(2, 10), silhouette, marker='s')
plt.xlabel('$k$')
plt.ylabel('silhouette_score')
plt.show()

---

Максимальное значение silhouette_score на графике показывает, какое количество кластеров необходимо строить.

---

Метрика [Калински-Харабаса](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html) рассматривает отношение дисперсии объектов внутри кластера и дисперсии между кластерами. Недостатком метрики является отсутствие граничного значения, поэтому необходимо посчитать метрику в соседних разбиениях по кластерам в $k-1$ и $k+1$. Достоинство метрики - быстрое вычисление:

In [ ]:
from sklearn.metrics import calinski_harabasz_score

print(calinski_harabasz_score(X, kmeans.labels_))

ch = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=21).fit(X)
    ch.append(calinski_harabasz_score(X, km.labels_))

plt.plot(range(2, 10), ch, marker='s')
plt.xlabel('$k$')
plt.ylabel('calinski_harabasz_score')
plt.show()

---

За счет скорости, метрика calinski_harabasz_score может быть лучшим способом определить количество кластеров.

---

Метрика [davies_bouldin_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html) так же оценивает близость объектов внутри кластера и удаленность от других кластеров. Чем ближе значение метрики к нулю - тем лучше.

In [ ]:
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(X, kmeans.labels_))

db = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=21).fit(X)
    db.append(davies_bouldin_score(X, km.labels_))

plt.plot(range(2, 10), db, marker='s')
plt.xlabel('$k$')
plt.ylabel('davies_bouldin_score')
plt.show()

---

Существуют и другие метрики, но нам для анализа достаточно рассмотренных. Все метрики можно собрать в одну функцию, а их результаты записывать в датафрейм:

In [ ]:
def metrics4(X, labels, y):
    result = [adjusted_rand_score(y, labels)]
    result.append(silhouette_score(X, labels, random_state=21))
    result.append(calinski_harabasz_score(X, labels))
    result.append(davies_bouldin_score(X, labels))
    return result

m4 = pd.DataFrame(index = ['adjusted_rand_score', 'silhouette_score', 'calinski_harabasz_score', 
                           'davies_bouldin_score'])
m4['KMeans'] = metrics4(X, kmeans.labels_, y)
m4

---

Метод к-средних удобен, прост, нагляден и легко интерпретируется. Недостатком метода является то, что он представляет из себя NP-сложную задачу и при больших объемах данных будет медленно работать.

---

# Задание 1

1. Загрузите данные из файла ***данные для кластеризации.xlsx*** (находится в папке datasets).
2. Метки класса из столбца *Office* присвойте вектору *y*, остальные данные для кластеризации присвойте матрице *X*.
3. Нормализуйте данные матрицы X.
4. Импортируйте метод ***KMeans***.
5. Проведите анализ на количество кластеров с помощью метода локтя, а так же на основе метрик *silhouette_score*, *calinski_harabasz_score* и *davies_bouldin_score*. 
6. Сделайте выводы по результатам анализа.
7. Обучите модель *KMeans* с разбиением на два кластера.
8. Оцените результат с помощью *classification_report()*, сравнив попадание объектов в кластеры с реальными классами объектов из исходного файла с данными. Имейте в виду, что кластеры - это не классы, поэтому важно то, что одни и те же объекты попали внутрь одной группы, а не то, какой номер группы им присвоен моделью. На протяжении всех заданий мы будем оценивать качество кластеризации, сравнивая кластеры с классификацией данных по признакам: Rain и Office.
9. Оцените результат кластеризации с помощью функции *metrics4()* и заполните соответствующий столбец таблицы итогов кластеризации *m4*.
10. Сделайте выводы по результатам оценки качества кластеризации.

---

***Рекомендации:***

1. При проведении анализа на количество кластеров выбирайте верхнюю границу диапазона не меньше 100 (для метода локтя 50).
2. Не забывайте, что модель может нумеровать кластеры не в том порядке, как в данных.

# ЗАДАНИЕ 1

---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pylab
import seaborn as sns
#plt.style.use('seaborn')
%matplotlib inline
pylab.rcParams['figure.figsize'] = (15, 10)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')   # чтобы можно было оттуда считать файл с данными для этого задания
df = pd.read_excel('/content/drive/MyDrive/data/данные_для_кластеризации.xlsx') # Создаем датафрейм df из файла
df 



In [ ]:
# y = df.Rain
# X = df.drop('Rain', axis=1)
# X

# y = df.Office
X = df.drop('Office', axis=1)
# X = X.drop('Rain', axis=1)
X
# Z = X.drop('Rain', axis=1)
Z = X.drop('weekday_cos', axis=1)
Z = Z.drop('Order_bac_cos', axis=1)
X
# Z

Z = (Z - Z.mean(axis=0))/Z.std(axis=0)

# Z['Rain'] = X.Rain
Z
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
# sc = StandardScaler()
# sc.fit(X_train)                                  
# X_train_std = sc.transform(X_train)              
# X_test_std = sc.transform(X_test)

# lr = LogisticRegression(random_state=21)
# lr.fit(X_train_std, y_train)                     
# y_pred = lr.predict(X_test_std)                  

# print(classification_report(y_test, y_pred))
# X = (X - X.mean(axis=0))/X.std(axis=0)



# X = (X - X.mean(axis=0))/X.std(axis=0)
# X


# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=2, random_state=21).fit(X)
# plt.scatter(X[:,2], X[:,4], c=kmeans.labels_)
# plt.show()

# from sklearn.datasets import make_blobs
# X, y = make_blobs(n_samples=1000, n_features=5, centers=4, random_state=21)

# X = (X - X.mean(axis=0))/X.std(axis=0)


In [ ]:
# df = pd.DataFrame(X)
# df['Office'] = y
# # y
# sns.pairplot(df, hue="Office", height=3)
# plt.plot()

df = pd.DataFrame(Z)
df['Office'] = y
# y
# sns.pairplot(df, hue="Office", height=3)
# plt.plot()

In [ ]:
# X = (X - X.mean(axis=0))/X.std(axis=0)
# X
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=2, random_state=21).fit(X)
# plt.scatter(X[:,2], X[:,4], c=kmeans.labels_)
# plt.show()

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=21).fit(Z)

# plt.scatter(X['Order_bac_sin'], X['Order_bac_cos'], c=kmeans.labels_)
plt.scatter(Z['Order_bac_sin'], Z['Rain'], c=kmeans.labels_)
plt.show()

In [ ]:
# from sklearn.cluster import KMeans

from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, kmeans.labels_)

0.6383875030533672

In [ ]:
inertia = []
for k in range(1, 4):
    kmeans = KMeans(n_clusters=k, random_state=21).fit(Z)
    inertia.append(np.sqrt(kmeans.inertia_))

plt.plot(range(1, 4), inertia, marker='s')
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_score(Z, kmeans.labels_, random_state=21)

0.2896711295462786

In [ ]:
silhouette = []
for k in range(2, 5):
    km = KMeans(n_clusters=k, random_state=21).fit(Z)
    silhouette.append(silhouette_score(Z, km.labels_, random_state=21))

plt.plot(range(2, 5), silhouette, marker='s')
plt.xlabel('$k$')
plt.ylabel('silhouette_score')
plt.show()

In [ ]:
from sklearn.metrics import calinski_harabasz_score

print(calinski_harabasz_score(Z, kmeans.labels_))

ch = []
for k in range(2, 5):
    km = KMeans(n_clusters=k, random_state=21).fit(Z)
    ch.append(calinski_harabasz_score(Z, km.labels_))

plt.plot(range(2, 5), ch, marker='s')
plt.xlabel('$k$')
plt.ylabel('calinski_harabasz_score')
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(Z, kmeans.labels_))

db = []
for k in range(2, 5):
    km = KMeans(n_clusters=k, random_state=21).fit(Z)
    db.append(davies_bouldin_score(Z, km.labels_))

plt.plot(range(2, 5), db, marker='s')
plt.xlabel('$k$')
plt.ylabel('davies_bouldin_score')
plt.show()

In [ ]:
# y = df.Office
# X = df.drop('Office', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
# sc = StandardScaler()
# sc.fit(X_train)                                  
# X_train_std = sc.transform(X_train)              
# X_test_std = sc.transform(X_test)

# lr = LogisticRegression(random_state=21)
# lr.fit(X_train_std, y_train)                     
# y_pred = lr.predict(X_test_std)  

# from sklearn.datasets import make_blobs
# # X
# X, y = make_blobs(n_samples=1000, n_features=5, centers=4, random_state=21)
# X, y
# X
# y
# X
# X = (X - X.mean(axis=0))/X.std(axis=0)
# X
# kmeans = KMeans(n_clusters=2, random_state=21).fit(X)
# plt.scatter(X[:,2], X[:,4], c=kmeans.labels_)
# plt.show()

array([[-8.13662364, -4.81444713,  3.9359552 , -8.21735083, -4.63344305],
       [-6.42268911, -6.20108463,  3.49246328, -8.62177893, -6.63212885],
       [-9.49480613, -2.71346591,  2.9051504 , -1.09377871,  1.7888751 ],
       ...,
       [-9.31503967, -4.80203821,  5.05315616, -4.47198345,  2.63358165],
       [-8.83400659, -4.23418288,  2.84614297, -2.73972074,  2.5464589 ],
       [-9.05640116,  6.35742895, -7.11944391, -7.10158446, -1.34642424]])

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
# X
# X2 = X.drop('weekday_sin', axis=1)
# X2 = X2.drop('weekday_cos', axis=1)
# X2 = X2.drop('Delivery_time_norm', axis=1)
# X2 = X2.drop('Rain', axis=1)
# X2 = X2.drop('Office', axis=1)
# X2
model_2 = AgglomerativeClustering(n_clusters=3).fit(Z)
print(classification_report(y, model_2.labels_))


              precision    recall  f1-score   support

           0       0.92      0.82      0.86      9389
           1       0.73      0.38      0.50     11402
           2       0.00      0.00      0.00         0

    accuracy                           0.58     20791
   macro avg       0.55      0.40      0.45     20791
weighted avg       0.81      0.58      0.66     20791



In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
# y
def metrics4(Z, labels, y):
    result = [adjusted_rand_score(y, labels)]
    result.append(silhouette_score(Z, labels, random_state=21))
    result.append(calinski_harabasz_score(Z, labels))
    result.append(davies_bouldin_score(Z, labels))
    return result

m4 = pd.DataFrame(index = ['adjusted_rand_score', 'silhouette_score', 'calinski_harabasz_score', 
                           'davies_bouldin_score'])
m4['KMeans'] = metrics4(Z, model_2.labels_, y)
m4

Вывод:2/3 clasters

---



In [ ]:
# повторим некоторые функции, которые пригодятся вам при решении задания

import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """Функция, которая строит график ошибок классификации TP, FP, TN, FN. Она нам понадобится
    для оценки качества кластеризации при сравнении кластеров с классами"""
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=3)
        thresh = 0.5   
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Class label')
    plt.xlabel('Cluster label')
    
def plot_dendrogram(model, **kwargs):
    """Функция, которая принимает модель агломеративной кластеризации и строит
    дендрограмму исследуемых данных"""
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
    hierarchy.dendrogram(linkage_matrix, **kwargs)

def metrics4(X, labels, y):
    """Функция, которая вычисляет различные метрики, которые мы будем использовать
    для оценки качества кластеризации. Здесь у - это колонка с реальными метками
    классов датасета"""
    result = [adjusted_rand_score(y, labels)]
    result.append(silhouette_score(X, labels, random_state=21))
    result.append(calinski_harabasz_score(X, labels))
    result.append(davies_bouldin_score(X, labels))
    return result

m4 = pd.DataFrame(index = ['adjusted_rand_score', 'silhouette_score', 'calinski_harabasz_score', 
                           'davies_bouldin_score'])

---

## Агломеративный метод

Агломеративный метод - один из методов иерархической кластеризации. Иерархическая кластеризация позволяет получить не просто разбиение на кластеры, а целую иерархию вложенных кластеров. Агломеративный метод изначально считает каждый объект отдельным классом, затем на каждом шаге он начинает объединять попарно ближайшие классы. Алгоритм прекращает работу, когда все объекты объединены в один класс.

Достоинства метода:
1. В результате имеем иерархию возможных разбиений на кластеры.
2. Удобная визуализация результатов в виде дендрограммы.

Недостатки метода:
1. Требует большие объемы памяти компьютера.
2. Медленно работает при больших объемах данных.

---

Ипортируем агломеративный методод [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) и [hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html) от куда нам понадобится функция [dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram) для визуализации дендрограммы.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

---

Напишем функцию подготовки данных и визуализации дендрограммы и обучим модель кластеризации агломеротивным методом:

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
    hierarchy.dendrogram(linkage_matrix, **kwargs)
    
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(Z)
plot_dendrogram(model, truncate_mode='level', p=3)

---

Количество кластеров в данных можно определить на основе дендрограммы - большие расстояния между кластерами и их объединением (ось ординат) свидетельствуют о различных кластерах. Анализируя полученную дендрограмму можно сделать вывод о наличии четырех кластеров.

Обучим модель непосредственно для 4 кластеров и оценим качество кластеризации:

In [ ]:
model_2 = AgglomerativeClustering(n_clusters=4).fit(X)
print(classification_report(y, model_2.labels_))

---

Анализ показывает, что мы угадали только 50% объектов, при этом в первом и четвертом классе мы не угадали верно ни одного объекта. 

Посмотрим визуализацию кластеризации:

In [ ]:
plt.scatter(X[:,2], X[:,4], c=model_2.labels_)
plt.show()

---

Визуальный анализ показывает, что все кластеры были верно предсказаны. Следовательно проблема в том, что метод просто нумерует кластеры не в той последовательности, что заложена изначально. Проверим это:

In [ ]:
for j in range(model_2.labels_.max()+1):
    print('метка класса =', j, '   метка кластера =', model_2.labels_[y==j].mean(), 
          '   стандартное отклонение меток кластера =', model_2.labels_[y==j].std())

---

Получается, что классу с меткой 0 соответствует кластер с меткой 3 и наоборот. Для дальнейшей оценки качества кластеризаци создадим вектор меток классов y_ac, соответствующий результатам кластеризации:

In [ ]:
y_ac = np.full(y.shape, fill_value=np.nan)
for j in range(4):
     y_ac[y == j] = round(model_2.labels_[y==j].mean(), 0)
print(classification_report(y_ac, model_2.labels_))

---

100% объектов классов угаданы при кластеризации.

Подсчитаем и запишем в таблицу метрики качества кластеризации:

In [ ]:
m4['AgglomerativeClustering'] = metrics4(X, model_2.labels_, y_ac)
m4

---

# Задание 2

1. Обучите общую модель агломеративной кластеризации.
2. Сделайте вывод о количестве кластеров в данных на основе дендрограммы.
3. Обучите модель агломеративной кластеризации для двух кластеров.
4. Оцените качество кластеризации и запишите результаты в таблицу.
5. Сделайте выводы о результатах кластеризации.

---

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
    hierarchy.dendrogram(linkage_matrix, **kwargs)
    
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(X)
plot_dendrogram(model, truncate_mode='level', p=3)

In [ ]:
model_2 = AgglomerativeClustering(n_clusters=2).fit(X)
print(classification_report(y, model_2.labels_))



              precision    recall  f1-score   support

           0       0.76      0.91      0.83      9389
           1       0.91      0.76      0.83     11402

    accuracy                           0.83     20791
   macro avg       0.84      0.84      0.83     20791
weighted avg       0.84      0.83      0.83     20791



In [ ]:
for j in range(model_2.labels_.max()+1):
    print('метка класса =', j, '   метка кластера =', model_2.labels_[y==j].mean(), 
          '   стандартное отклонение меток кластера =', model_2.labels_[y==j].std())

метка класса = 0    метка кластера = 0.08744275215677921    стандартное отклонение меток кластера = 0.2824827733721603
метка класса = 1    метка кластера = 0.7633748465181547    стандартное отклонение меток кластера = 0.4250102236670766


In [ ]:
y_ac = np.full(y.shape, fill_value=np.nan)
for j in range(4):
     y_ac[y == j] = round(model_2.labels_[y==j].mean(), 0)
print(classification_report(y_ac, model_2.labels_))

In [ ]:
m4['AgglomerativeClustering'] = metrics4(X, model_2.labels_, y_ac)
m4

## Метод главных компонент

Метод главных компонент [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) позволяет с помощью линейных преобразований исходных признаков перейти в новое признаковое пространство. В результате преобразований появляется возможность снизить количество признаков при минимальной потере дисперсии. Количество главных компонент опеределяется исследователем на основании требования к сохраненной дисперсии. Обычно ориентируются на 90% дисперсии, но, исходя из конкретики задачи, требование к сохраненной дисперии может быть и больше и меньше. Например, при сильном шуме в данных, требование к сохраненной дисперсии может быть значительно ниже 90%. Порог сохраненной дисперсии может определяться эксперементально, на основе качества решения итоговой задачи.

Достоинства метода главных компонент:
1. Главные компоненты ортогональны друг другу, что позволяет решить проблему мультиколлинеарности признаков.
2. В случае большого количества исходных признаков, позволяет снизить вычислительную нагрузку в задачах классификации и регресии с незначительной потерей качества результата.
3. За счет отбрасывания части главных компонент, позволяет снизить шум в данных, что снижает эффект переобучения моделей.

Недостатки метода:
1. Плохо работает в случае низкой корреляции среди исходных признаков.
2. Плохо работает для визуализации многомерных пространств признаков на плоскости.

---

Импортируем метод PCA и обучим модель для 5 главных компонент:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5, random_state=21)
X_centered = X - X.mean(axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)

---

Визуализируем накопленную сохраненную димперсию:

In [ ]:
plt.plot(np.arange(1,6), np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.xlim(1, 6)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axhline(0.9, c='r')
plt.show()

In [ ]:
print('Первые две главных компоненты сохраняют', str(round(100*pca.explained_variance_ratio_[:2].sum(),2))+'%', 
      'дисперсии, три компоненты -', str(round(100*pca.explained_variance_ratio_[:3].sum(),2))+'%')

---

С целью снижения размерности данных имеет смысл оставлять три главных компоненты. Однако, в нашей задаче и две главных компоненты выполняют свою задачу по разделению кластеров:

In [ ]:
pca = PCA(n_components=2, random_state=21)
X_centered = X - X.mean(axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y)
plt.show()

---

Однако два кластера оказались достаточно близко друг к другу. При подготовке этого ноутбука, данные многократно обновлялись, и на многих из них кластеры пересекались.

---

# Задание 3

1. Обучить модель для 6 главных компонент.
2. Визуализировать сохраненную дисперсию.
3. Определить количество главных компонент, сохраняющих не менее 90% дисперсии.
4. Обучить модель для двух главных компонент и визуализировать данные.
5. Сделать выводы об эффективности применения метода главных компонент к решаемой задаче. По какому признаку можно разделить выборку на кластеры?

---

## t-SNE

[t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) - еще один метод снижения размерности. В отличие от метода главных компонент, t-SNE сжимает данные таким образом, чтобы сохранить их структуру. При этом полученное расстояние в сжатом признаковом в пространстве скорее всего никак не будет соотносится с расстояниями в исходном признаковом в пространстве. Метод пытается перенести окрестность каждой точки из исходного пространства в сжатое. 

Достоинства метода:
1. Метод прост в применении.
2. Работает с большими объемами данных.
3. Эффективен при визуализации на плоскости многомерных данных.

Недостатки метода:
1. Полученные результаты практически не интерпретируются.
2. Может найти структуру в абсолютном шуме.

---

Импортируем метод t-SNE:

In [ ]:
from sklearn.manifold import TSNE

Обучим модели для двух компонент, с гиперпараметром ***perplexity*** равным 10 и 100 и визуализируем результаты:

In [ ]:
tsne = TSNE(n_components=2, perplexity=10, random_state=21)
tsne_representation = tsne.fit_transform(X)
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1])
plt.title('perplexity = 10')
plt.show()

tsne = TSNE(n_components=2, perplexity=100, random_state=21)
tsne_representation = tsne.fit_transform(X)
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1])
plt.title('perplexity = 100')
plt.show()

---

Увеличение параметра perplexity приводит к уменьшению расстояния между точками.

Визуализируем принадлежность к кластерам при двух компонентах и perplexity=25:

In [ ]:
tsne = TSNE(n_components=2, perplexity=25, random_state=21)
tsne_representation = tsne.fit_transform(X)
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], c=y)
plt.title('perplexity = 25')
plt.show()

---

# Задание 4

Визуализируйте данные с помощью t-SNE при различных значениях perplexity. По какому признаку можно разделить данные на кластеры при достаточно большом значении perplexity?

---

## DBSCAN

Еще один метод кластеризации - [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

Достоинства метода:

1. Самостоятельно определяет количество кластеров в зависимости от параметра eps.
2. Кроме кластеров определяет шум.
3. Работает с кластерами разной формы.

Недостатки метода:

1. С помощью метода невозможно проверить гипотезу о количестве кластеров, т.к. метод определяет их самостоятельно.
2. Необходим поиск параметра eps.
3. Не работает, если кластеры разной плотности.

Импортируем метод:

In [ ]:
from sklearn.cluster import DBSCAN

---

Для отображения результатов на плоскости снизим размерность задачи с помощью t-SNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=15, random_state=21)
tsne_X = tsne.fit_transform(X)

---

Обучим модель с параметрами по умолчанию и напишем функцию визуализации результатов кластеризации:

In [ ]:
dbscan = DBSCAN(eps=0.5)
dbscan.fit(X)

def viz_dbscan(labels, x2d):
    clstr = []
    k = labels.min()
    if k == -1:
        title = ['DBSCAN нашел ', ' кластера и шум']
    else:
        title = ['DBSCAN нашел ', ' кластера']
    for j in range(k, labels.max()+1):
        plt.scatter(x2d[labels==j, 0], x2d[labels==j, 1])
        if j == -1:
            clstr = ['Шум']
        else:
            clstr.append('Кластер '+str(j+1))
    plt.legend(clstr)
    plt.title(title[0] + str(len(clstr)+k) + title[1])
    plt.show()
    
viz_dbscan(dbscan.labels_, tsne_X)

---

Кластеры определены верно, но найден шум, который в принципе необходимо удалить. Попробуем изменить параметр ***eps*** модели. В этом нам поможет [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html). Установим количество ближайших соседей *n_neighbors=25*, это означает что в кластере должно быть не менее 25 объектов.

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model_NN = NearestNeighbors(n_neighbors=25)
model_NN.fit(X)
dist, _ = model_NN.kneighbors(X, n_neighbors=25, return_distance=True)
dist = np.sort(dist[:,-1])
plt.plot(dist)
plt.plot([0,1000], [0.8, 0.8], 'r')
plt.show()

---

Параметр ***eps*** по оси ординат необходимо выбрать так, чтобы после пересечения прямой с графиком, график шел вертикально вверх. Построим модель с eps=0.8:

In [ ]:
dbscan = DBSCAN(eps=0.8)
dbscan.fit(X)
viz_dbscan(dbscan.labels_, tsne_X)

---

DBSCAN верно определил кластеры и не нашел шума.

---

Оценим качество кластеризации:

In [ ]:
for j in range(dbscan.labels_.max()+1):
    print('метка класса =', j, '   метка кластера =', dbscan.labels_[y==j].mean(), 
          '   стандартное отклонение меток кластера =', dbscan.labels_[y==j].std())

In [ ]:
y_ac = np.full(y.shape, fill_value=np.nan)
for j in range(4):
     y_ac[y == j] = int(dbscan.labels_[y==j].mean())
print(classification_report(y_ac, dbscan.labels_))

In [ ]:
m4['DBSCAN'] = metrics4(X, dbscan.labels_, y_ac)
m4

---

Все методы показали одинаковые значения метрик качества кластеризации, в силу того, что продемонстрировали одинаковые результаты.

---

Можно посмотреть, при каких значениях eps сколько кластеров строит DBSCAN. Например, следующий код строит зависимость количества кластеров от eps, изменяющегося от 0.1 до 0.99:

In [ ]:
epsilon = []
n_claster = []
for j in range(10, 100, 1):
    dbscan = DBSCAN(eps=j/100)
    dbscan.fit(X)
    epsilon.append(j/100)
    n_claster.append(dbscan.labels_.max()+1)
plt.plot(epsilon, n_claster, marker='s')
plt.show()

---

И можно посмотреть, как DBSCAN разбивает наши данные на 19 кластеров (eps=0.18):

In [ ]:
dbscan = DBSCAN(eps=0.18)
dbscan.fit(X)
viz_dbscan(dbscan.labels_, tsne_X)

---

Для оценки качества полученного разбиения не обязательно применять метрики, достаточно взглянуть на процент данных, определенных как шум:

In [ ]:
print('DBSCAN определил', str(len(dbscan.labels_[dbscan.labels_==-1])/10) +'%', 'данных как шум')

DBSCAN определил 70.8% данных как шум


---

# Задание 5

1. Заголовок последнего графика содержит ошибку "19 кластера". Необходимо написать функцию ***add_a_ov(n, word)***, которая поможет исправить ошибку. Функция обладает следующими характеристиками:
    - Функция получает на вход два параметра n и word:
        - n - числительное в виде целого числа; 
        - word - существительное мужского рода в единственном числе в именительном падеже.
    - Функция возвращает измененное word в соответствии с числительным n.
    - Примеры выполнения функции:
        - *add_a_ov(1, 'кластер')* вернет 'кластер';
        - *add_a_ov(2, 'кластер')* вернет 'кластера';
        - *add_a_ov(5, 'кластер')* вернет 'кластеров'.
        
        
2. Внесите изменения в функцию ***viz_dbscan()*** с учетом реализации функции ***add_a_ov(n, word)***.
3. Обучите модель DBSCAN с параметрами по умолчанию и визуализируйте результат.
4. Проведите поиск гиперпараметра ***eps***.
5. Проведите визуализацию зависимости количества кластеров от параметра ***eps***.
6. Обучите модель DBSCAN с таким параметром ***eps***, чтобы получить разбиение на два кластера.
7. Визуализируйте результат.
8. Проведите анализ соответствия полученных кластеров классам.
9. Оцените качество кластеризации и запишите результаты в таблицу.
10. Сделайте общий вывод по результатам кластеризации различными методами.

In [ ]:
def add_a_ov(n, word):
  _ret = ""
  if n % 100 > 10 and n % 100 < 20:
    return word + "ов"
  if n % 10 == 1:
    return word
  elif n % 10 > 1 and n % 10 < 5:
    return word + "а"
  else:
    return word + "ов"

dbscan = DBSCAN(eps=0.5)
dbscan.fit(X)

def viz_dbscan(labels, x2d):
    clstr = []
    k = labels.min()
    print(k)
    if k == -1:
        title = ['DBSCAN нашел ', ' ' + add_a_ov(labels.max() + 1, 'кластер') + ' и шум']
    else:
        title = ['DBSCAN нашел ', ' ' + add_a_ov(labels.max() + 1, 'кластер')]
    for j in range(k, labels.max()+1):
        plt.scatter(x2d[labels==j, 0], x2d[labels==j, 1])
        if j == -1:
            clstr = ['Шум']
        else:
            clstr.append('Кластер '+str(j+1))
    plt.legend(clstr)
    plt.title(title[0] + str(len(clstr)+k) + title[1])
    plt.show()
    
viz_dbscan(dbscan.labels_, tsne_X)

In [ ]:
# код проверки правильности функции add_a_ov. Запустите его, чтобы узнать, правильно ли описана функция

def test(n, word):
    n = n%100
    if n%10 in [2,3,4] and n//10 != 1:
        result = word + 'а'
    elif n%10 == 1 and n != 11:
        result = word
    else:
        result = word + 'ов'
    return result


for j in [i for i in range(31)]+[101,102,105,111,112,1001,1002,1005,1011,1012,1021,1022]:
    if test(j, 'слон') != add_a_ov(j, 'слон'):
        break
if j < 1022:
    print(j, add_a_ov(j, 'слон'))
    if j < 10:
        print('Функция содержит критичные ошибки')
    elif j < 20:
        print('Функция содержит критичные ошибки. Не верно обрабатываются числительные от 11 до 14')
    elif j < 100:
        print('Функция содержит критичные ошибки. Не верно обрабатываются числительные от 20 до 100')
    elif j < 1000:
        print('Функция содержит не критичные для задачи ошибки.', 
              'Не верно обрабатываются числительные свыше 100')
    elif j >= 1000:
        print('Функция содержит не критичные для задачи ошибки.', 
              'Не верно обрабатываются числительные от 1000 и выше')
else:
    print('Функция работает нормально')

In [ ]:
from sklearn.cluster import DBSCAN

tsne = TSNE(n_components=2, perplexity=15, random_state=21)
tsne_X = tsne.fit_transform(X)

dbscan = DBSCAN(eps=0.5)
dbscan.fit(X)

def viz_dbscan(labels, x2d):
    clstr = []
    k = labels.min()
    if k == -1:
        title = ['DBSCAN нашел ', ' кластера и шум']
    else:
        title = ['DBSCAN нашел ', ' кластера']
    for j in range(k, labels.max()+1):
        plt.scatter(x2d[labels==j, 0], x2d[labels==j, 1])
        if j == -1:
            clstr = ['Шум']
        else:
            clstr.append('Кластер '+str(j+1))
    plt.legend(clstr)
    plt.title(title[0] + str(len(clstr)+k) + title[1])
    plt.show()
    
viz_dbscan(dbscan.labels_, tsne_X)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_NN = NearestNeighbors(n_neighbors=25)
model_NN.fit(X)
dist, _ = model_NN.kneighbors(X, n_neighbors=25, return_distance=True)
dist = np.sort(dist[:,-1])
plt.plot(dist)
plt.plot([0,1000], [0.8, 0.8], 'r')
plt.show()

In [ ]:
dbscan = DBSCAN(eps=0.8)
dbscan.fit(X)
viz_dbscan(dbscan.labels_, tsne_X)

In [ ]:
for j in range(dbscan.labels_.max()+1):
    print('метка класса =', j, '   метка кластера =', dbscan.labels_[y==j].mean(), 
          '   стандартное отклонение меток кластера =', dbscan.labels_[y==j].std())

In [ ]:
y_ac = np.full(y.shape, fill_value=np.nan)
for j in range(4):
     y_ac[y == j] = int(dbscan.labels_[y==j].mean())
print(classification_report(y_ac, dbscan.labels_))

In [ ]:
m4['DBSCAN'] = metrics4(X, dbscan.labels_, y_ac)
m4

In [ ]:
epsilon = []
n_claster = []
for j in range(10, 100, 1):
    dbscan = DBSCAN(eps=j/100)
    dbscan.fit(X)
    epsilon.append(j/100)
    n_claster.append(dbscan.labels_.max()+1)
plt.plot(epsilon, n_claster, marker='s')
plt.show()

In [ ]:
dbscan = DBSCAN(eps=0.165)
dbscan.fit(X)
viz_dbscan(dbscan.labels_, tsne_X)

In [ ]:
print('DBSCAN определил', str(len(dbscan.labels_[dbscan.labels_==-1])/10) +'%', 'данных как шум')

DBSCAN определил 98.9% данных как шум


In [ ]:
for j in range(dbscan.labels_.max()+1):
    print('метка класса =', j, '   метка кластера =', dbscan.labels_[y==j].mean(), 
          '   стандартное отклонение меток кластера =', dbscan.labels_[y==j].std())

In [ ]:
y_ac = np.full(y.shape, fill_value=np.nan)
for j in range(4):
     y_ac[y == j] = int(dbscan.labels_[y==j].mean())
print(classification_report(y_ac, dbscan.labels_))

In [ ]:
m4['DBSCAN'] = metrics4(X, dbscan.labels_, y_ac)
m4